In [17]:
from time import time
import configparser
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
from IPython.display import display, HTML



In [2]:
%load_ext sql

# Get the params of the created redshift cluster and establish a database connection.

In [11]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()


# Sample queries.

## 1 .Queries to get count of number of records in each table.

In [12]:
staging_events_1="select count(*) from sparkifydb.staging_events;"
staging_songs_1="select count(*) from sparkifydb.staging_songs;"
users_1="select count(*) from sparkifydb.users;"
artists_1="select count(*) from sparkifydb.artists;"
songs_1="select count(*) from sparkifydb.songs;"
songplays_1="select count(*) from sparkifydb.songplay;"
time_1="select count(*) from sparkifydb.time;"


In [22]:
def getNumberofRecords():
    ''' This function executes all the SQL queries that counts number
    of records from tables and displays all results in dataframe.'''
    
    queryResults  =[] 
    for i,query in enumerate([staging_events_1,staging_songs_1,users_1,artists_1,songs_1,songplays_1,time_1]):
        cur.execute(query)
        results = cur.fetchone()
        queryResults.append(results)
    return pd.DataFrame({"Table":["staging_events","staging_songs","users","artists","songs","songplay","time"], "Records":queryResults})#.set_index('query')

In [23]:
tableCount=getNumberofRecords()

In [24]:
display(HTML(tableCount.to_html()))

,Table,Records
0,staging_events,"(8056,)"
1,staging_songs,"(14896,)"
2,users,"(104,)"
3,artists,"(10025,)"
4,songs,"(14896,)"
5,songplay,"(333,)"
6,time,"(6813,)"


## 2. Queries to show first 10 records from each table to check if data inserted correctly.

In [44]:
staging_events="select * from sparkifydb.staging_events LIMIT 10;"
staging_songs="select * from sparkifydb.staging_songs LIMIT 10;"
users="select * from sparkifydb.users LIMIT 10;"
artists="select * from sparkifydb.artists LIMIT 10;"
songs="select * from sparkifydb.songs LIMIT 10;"
songplays="select * from sparkifydb.songplay LIMIT 10;"
time="select * from sparkifydb.time LIMIT 10;"



In [50]:
def getRecords():
     ''' This function executes all the SQL queries that selects 10 records from
     every table and displays in the dataframe.'''
    
    queryResults  =[] 
    for i,query in enumerate([staging_events,staging_songs,users,artists,songs,songplays,time]):
        print("#################################################################")
        print("Query:"+query)
        print("#################################################################")
        df= pd.read_sql_query(query, conn)
        display(HTML(df.to_html()))

In [51]:
getRecords()

#################################################################
Query:select * from sparkifydb.staging_events LIMIT 10;
#################################################################


,event_id,artist_name,auth,firstname,gender,iteminsession,lastname,length,level,location,method,page,registration,sessionid,song,status,ts,useragent,userid
0,7,Miami Horror,Logged In,Kate,F,88,Harrell,250.82730,paid,"Lansing-East Lansing, MI",PUT,NextSong,1540472624796,293,Sometimes,200,1541548876796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
1,11,Nirvana,Logged In,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1541022995796,237,Serve The Servants,200,1541381242796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
2,28,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,33,Lifehouse,Logged In,Jahiem,M,2,Miles,203.59791,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1540817347796,42,We'll Never Know,200,1541300337796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
4,41,Olivia Ruiz,Logged In,Jahiem,M,3,Miles,254.74567,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1540817347796,42,Cabaret Blanco,200,1541300540796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
5,42,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,47,BjÃÂ¶rk,Logged In,Kate,F,93,Harrell,348.57751,paid,"Lansing-East Lansing, MI",PUT,NextSong,1540472624796,293,Undo,200,1541550104796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
7,55,Kanye West,Logged In,Kate,F,94,Harrell,198.47791,paid,"Lansing-East Lansing, MI",PUT,NextSong,1540472624796,293,Celebration,200,1541550452796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
8,59,Maroon 5,Logged In,Aleena,F,6,Kirby,173.66159,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1541022995796,237,Harder To Breathe,200,1541382480796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
9,66,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


#################################################################
Query:select * from sparkifydb.staging_songs LIMIT 10;
#################################################################


,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARNHTE41187B99289A,None,None,,Saafi Brothers,SOAACFC12A8C140567,Supernatural Pt. II,343.09179,0
1,1,AR3FKJ61187B990357,27.946529999999999,-82.459270000000004,"Tampa, FL",David Sanborn,SOAADJH12AB018BD30,Black Light (Album Version),385.90649,1975
2,1,AR2IKF71187FB4D0C2,None,None,,Ace of Base,SOAAMWQ12A8C144DF1,Happy Nation,255.08526,1992
3,1,ARAT5YT1187B9AE90B,None,None,"Miami, FL",Chris Smither,SOABVWY12AB018274B,Honeysuckle Dog,245.99465,1993
4,1,ARQFV881187FB3C24C,51.548810000000003,-0.060400000000000002,"Hackney, East London, England",Adrian Legg,SOACBGF12AC9097E79,O'Malley & Delacey,220.02893,0
5,1,ARCLPWN11F4C83DF42,None,None,,Omar And The Howlers,SOACMKO12A8C1448C1,Border Girl,232.88118,0
6,1,ARR3Y511187B9B2932,51.506320000000002,-0.12714,"London, England",A Man Called Adam,SOADFRL12AB01858A6,Techno Powers,514.27220,1997
7,1,ARVG8YI1187FB4CBA7,None,None,"Covington, Kentucky",Adrian Belew,SOADFSR12AB018C17E,Gunman,233.40363,1990
8,1,AR3K1T51187FB4042A,None,None,"Kannapolis, NC",George Clinton,SOADMFM12A8C131911,Hard As Steel,309.47220,0
9,1,ARQ35ML1187B99BB8D,None,None,,Ella Washington,SOADZEU12AB018AAFD,This Must Be Love,168.64608,0


#################################################################
Query:select * from sparkifydb.users LIMIT 10;
#################################################################


...

,song_id,artist_id,year,duration,level
0,SOAAAQN12AB01856D3,ARAMIDF1187FB3D8D4,0,153.36444,None
1,SOAACFC12A8C140567,ARNHTE41187B99289A,0,343.09179,None
2,SOAACTC12AB0186A20,ARXWFZ21187FB43A0B,2008,180.76689,None
3,SOAADAD12A8C13D5B0,ARQTC851187B9B03AF,2005,263.99302,None
4,SOAADJH12AB018BD30,AR3FKJ61187B990357,1975,385.90649,None
5,SOAADUU12AB0183B6F,AR70XXH1187FB44B55,0,101.92934,None
6,SOAAEHR12A6D4FB060,AR5N8VN1187FB37A4E,1991,415.81669,None
7,SOAAETA12A6D4FC626,ARQXK0B1187B9ACC97,2007,448.23465,None
8,SOAAFHQ12A6D4F836E,AR3CQ2D1187B9B1953,2006,322.84689,None
9,SOAAFUV12AB018831D,AR5ZGC11187FB417A3,0,216.05832,None


#################################################################
Query:select * from sparkifydb.songplay LIMIT 10;
#################################################################


,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,6,2018-11-05 15:43:22.796,97,paid,SODHZVG12A8C1404DD,ARS5WKC1187B9AC7D1,147,Maine,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
1,14,2018-11-09 14:50:53.796,15,paid,SOKMBHF12A58A7ACE4,ARQEL841187FB52D01,362,"Memphis, TN","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
2,22,2018-11-30 10:30:53.796,36,paid,SOOIPWS12AB018CBBB,ARKJS6R1187FB46324,998,"Detroit, MI","""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537...."
3,23,2018-11-24 11:19:06.796,80,paid,SOAIUDT12AF729F399,AR87P0Q1187B989ADF,903,"Orange County, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
4,30,2018-11-03 01:12:26.796,53,free,SOSELMV12A6D4FCF5A,ARWD25M1187FB4C563,52,London,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
5,31,2018-11-13 20:13:55.796,95,paid,SOHTQAS12A6701C7BA,AR12F2S1187FB56EEF,411,"Boston, MA","""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like..."
6,39,2018-11-15 16:40:02.796,49,paid,SOLQSYZ12A58A7919B,ARQSM561187FB4A0CF,621,"Orange County , USA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...
7,47,2018-11-14 05:04:27.796,80,paid,SOSLZJM12A58A7CF4E,ARQSM561187FB4A0CF,548,"Orange County , USA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
8,55,2018-11-14 15:14:54.796,25,paid,SOVXAXI12A8C1383D9,ARY65231187FB46283,534,,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
9,65,2018-11-07 18:11:11.796,15,paid,SODTRKW12A6D4F9A51,AR71MIY1187B9BA0C3,221,,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."


#################################################################
Query:select * from sparkifydb.time LIMIT 10;
#################################################################


,start_time,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,4
1,2018-11-01 21:05:52.796,21,1,44,11,2018,4
2,2018-11-01 21:08:16.796,21,1,44,11,2018,4
3,2018-11-01 21:11:13.796,21,1,44,11,2018,4
4,2018-11-01 21:17:33.796,21,1,44,11,2018,4
5,2018-11-01 21:24:53.796,21,1,44,11,2018,4
6,2018-11-01 21:28:54.796,21,1,44,11,2018,4
7,2018-11-01 21:42:00.796,21,1,44,11,2018,4
8,2018-11-01 21:52:05.796,21,1,44,11,2018,4
9,2018-11-01 21:55:25.796,21,1,44,11,2018,4


## 3. Query the songplay table to get the artist names of top 5 frequently played song.

In [75]:
# Select the most played song ID and the artist name corresponding to that song"

most_played_song_ID="SELECT s.song_id,COUNT(s.song_id) AS Frequency,s.artist_id, a.artist_name from sparkifydb.songplay as s JOIN sparkifydb.artists as a ON(a.artist_id=s.artist_id) group by s.song_id,s.artist_id,a.artist_name order by Frequency DESC LIMIT 5;"
df= pd.read_sql_query(most_played_song_ID, conn)
display(HTML(df.to_html()))



,song_id,frequency,artist_id,artist_name
0,SOBONKR12A58A7A7E0,37,AR5E44Z1187B9A1D74,Dwight Yoakam
1,SOUNZHU12A8AE47481,9,AR37SX11187FB3E164,Ron Carter
2,SOHTKMO12AB01843B0,9,AR5EYTL1187B98EDA0,Lonnie Gordon
3,SOULTKQ12AB018A183,8,ARKQQZA12086C116FC,B.o.B
4,SOLZOBD12AB0185720,6,ARPDVPJ1187B9ADBE9,Usher featuring Jermaine Dupri


## Verify if the results obtained above

In [76]:

verifyQuery="SELECT s.song_id,a.artist_id,a.artist_name FROM sparkifydb.songs as s JOIN sparkifydb.artists as a ON(a.artist_id=s.artist_id) WHERE s.song_id IN('SOBONKR12A58A7A7E0','SOUNZHU12A8AE47481','SOHTKMO12AB01843B0','SOULTKQ12AB018A183','SOLZOBD12AB0185720');"

df= pd.read_sql_query(verifyQuery, conn)
display(HTML(df.to_html()))



,song_id,artist_id,artist_name
0,SOBONKR12A58A7A7E0,AR5E44Z1187B9A1D74,Dwight Yoakam
1,SOHTKMO12AB01843B0,AR5EYTL1187B98EDA0,Lonnie Gordon
2,SOLZOBD12AB0185720,ARPDVPJ1187B9ADBE9,Usher featuring Jermaine Dupri
3,SOULTKQ12AB018A183,ARKQQZA12086C116FC,B.o.B
4,SOUNZHU12A8AE47481,AR37SX11187FB3E164,Ron Carter
